In [ ]:
!!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124
!pip install transformers
!pip install gradio
!pip install Pillow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.9/322.9 kB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 129.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 6.7 MB/s eta 0:00:00


In [ ]:
!pip install transformers accelerate

In [ ]:
import torch
print(torch.version.cuda)  # Check CUDA version PyTorch supports karne ke liye likha
print(torch.cuda.is_available())  # Should return True if GPU is detected hai to
print(torch.backends.cudnn.enabled)  # Should be True if cuDNN is working hai to

12.4
True
True


In [ ]:
!pip install gradio transformers torch pillow accelerate gtts

In [ ]:
!pip install gradio transformers torch pillow accelerate gtts

In [ ]:
!pip install SpeechRecognition

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 39.1 MB/s eta 0:00:00


In [ ]:
import gradio as gr
from transformers import BlipProcessor, BlipForConditionalGeneration
from PIL import Image
import torch
import re
from gtts import gTTS
import os
import speech_recognition as sr


device = "cuda" if torch.cuda.is_available() else "cpu"
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-large")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-large").to(device)
recognizer = sr.Recognizer()

def clean_caption(text):
    """ Repeated words ko remove karta hai aur better formatting karta hai """
    words = text.split()
    cleaned_words = []
    for word in words:
        if not cleaned_words or word.lower() != cleaned_words[-1].lower():
            cleaned_words.append(word)
    return " ".join(cleaned_words)

def speech_to_text(audio):
    """ Audio file ko text me convert karta hai """
    try:
        with sr.AudioFile(audio) as source:
            audio_data = recognizer.record(source)
            text = recognizer.recognize_google(audio_data)
            return text
    except Exception as e:
        return " Speech recognition failed! Try again."

def generate_caption(image, user_prompt="", audio=None):
    """ Image ko analyze karke meaningful aur detailed caption generate karta hai """
    if image is None:
        return " Please upload an image!", None

    image = Image.open(image).convert("RGB")


    if not user_prompt.strip() and audio is not None:
        user_prompt = speech_to_text(audio)

    prompt_text = user_prompt if user_prompt.strip() else "Describe the image in detail."
    inputs = processor(image, text=prompt_text, return_tensors="pt").to(device)


    output = model.generate(**inputs, temperature=0.7, top_p=0.9, max_length=100, repetition_penalty=1.2)


    caption = processor.decode(output[0], skip_special_tokens=True)
    caption = clean_caption(caption)


    tts = gTTS(text=caption, lang='hi')
    audio_path = "caption_audio.mp3"
    tts.save(audio_path)

    return caption, audio_path


app = gr.Interface(
    fn=generate_caption,
    inputs=[
        gr.Image(type="filepath", label=" Upload Image"),
        gr.Textbox(lines=2, placeholder=" Ask anything about the image...", label="Your Question"),
        gr.Audio(type="filepath", label=" Speak your question")
    ],
    outputs=[
        gr.Textbox(label=" AI's Answer"),
        gr.Audio(label=" AI Speech Output")
    ],
    title=" AI Image Captioning with BLIP-2 (Text & Audio Input)",
    description="Upload an image and ask anything about it via text or voice. AI will analyze and respond accordingly with a detailed answer and audio output.",
    theme="Huggingface-",)


app.launch(share=True)


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/445 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/527 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.60k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.88G [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/gradio/blocks.py:1116: UserWarning: Cannot load Huggingface-. Caught Exception: Repo id must use alphanumeric chars or '-', '_', '.', '--' and '..' are forbidden, '-' and '.' cannot start or end the name, max length is 96: 'Huggingface-'.
  warnings.warn(f"Cannot load {theme}. Caught Exception: {str(e)}")


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://54bc585a284dac44f2.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
